This script runs through some simple examples of how to use the prtype functions to create precipitation type data arrays based on a combination of a pysteps blended nowcast and snow level, temperature, and ground temperature data taken from a different model, such as INCA or COSMO. In these examples, INCA data is used.

In [1]:
# Import the required functions from prtype, and the Importer designed to import the files of your model
# in this case we use the example GRIB importer provided

from pysteps_postprocessor_diagnostics_prtype.postprocessor.postprocessor_diagnostics_prtype import *
from example_importer import *
import datetime

Pysteps configuration file found at: C:\Users\josep\anaconda3\Lib\site-packages\pysteps\pystepsrc



Define the input parameters

In [2]:
# NetCDF4 file path
filename = "insert path and filename of the netCDF file here"

# Start date and time of the data
startdate = datetime.datetime.strptime('202201080000', "%Y%m%d%H%M")

# INCA topography file path
# Note that the topography file should be a DEM in text format, only including the array data
topo = "insert path and filename of the topography file here"

# The projection string for the pySTEPS data
nwc_projection_string = 'insert the nowcast projection string here'

# The number of members to plot over time
members = 2

# The base length of time over which to plot
timeBase = 60

# The length of the time steps
timeStep = 5

# Choose whether we require the topographical file to be interpolated
topo_interpolation = False

Import the snow level, temperature, ground temperature, and model metadata from the INCA files

In [ ]:
# A list of the names of the GRIB import selection keys
keys = ['Nx', 'Ny', 'latitudeOfFirstGridPointInDegrees', 'longitudeOfFirstGridPointInDegrees', 'earthIsOblate',
        'LoVInDegrees', 'DxInMetres', 'DyInMetres', 'iScansNegatively', 'jScansPositively', 'Latin1InDegrees',
        'Latin2InDegrees']

In [3]:
# Import the snow level data
importer = IncaGribImporter()
incaDictionary_ZS = importer.retrieve_grib_data(filename=r"insert path and filename of snow level grb file here", metadata_keys=keys)

# Transform to a 3D matrix
snowLevelData = importer.inca_dictionary_to_3Dmatrix(incaDictionary_ZS)
del importer

In [4]:
# Import the temperature data
importer = IncaGribImporter()
incaDictionary_TT = importer.retrieve_grib_data(filename=r"insert path and filename of temperature grb file here", metadata_keys=keys)

# Transform to a 3D matrix
temperatureData = importer.inca_dictionary_to_3Dmatrix(incaDictionary_TT)

# Transform Kelvin to Celsius
temperatureData[:, :, :] = temperatureData[:, :, :] - 273.15
del importer

In [5]:
# import the ground temperature data
importer = IncaGribImporter()
incaDictionary_TG = importer.retrieve_grib_data(filename=r"insert path and filename of ground temperature grb file here", metadata_keys=keys)

# Transform to a 3D matrix
groundTemperatureData = importer.inca_dictionary_to_3Dmatrix(incaDictionary_TG)

# Transform Kelvin to Celsius
groundTemperatureData[:, :, :] = groundTemperatureData[:, :, :] - 273.15

del importer

In [7]:
print(incaDictionary_ZS)

{'Metadata': {'Nx': 601, 'Ny': 591, 'latitudeOfFirstGridPointInDegrees': 47.896, 'longitudeOfFirstGridPointInDegrees': 0.492, 'earthIsOblate': 1, 'LoVInDegrees': 4.359, 'DxInMetres': 1000, 'DyInMetres': 1000, 'iScansNegatively': 0, 'jScansPositively': 1, 'Latin1InDegrees': 49.833, 'Latin2InDegrees': 51.167}, 'Messages': {1: {'Attributes': {}, 'Grid': array([[   0.,    0.,    0., ...,  106.,  106.,  106.],
       [   0.,    0.,    0., ...,  106.,  106.,  107.],
       [   0.,    0.,    0., ...,  107.,  107.,  107.],
       ...,
       [1031., 1030., 1029., ...,  239.,  235.,  231.],
       [1033., 1032., 1031., ...,  237.,  233.,  229.],
       [1035., 1034., 1033., ...,  235.,  231.,  227.]])}, 2: {'Attributes': {}, 'Grid': array([[   0.,    0.,    0., ...,  103.,  104.,  105.],
       [   0.,    0.,    0., ...,  102.,  102.,  103.],
       [   0.,    0.,    0., ...,  102.,  102.,  102.],
       ...,
       [1029., 1028., 1027., ...,  233.,  229.,  226.],
       [1030., 1029., 1028., .

Define the model metadata. This can be extracted from your model data or defined directly

In [6]:
projection_bounds = "insert model projection bounds here in the form [x1, y1, x2, y2]"

modelMetadataDictionary = {'projection': 'insert model projection string here', 'xpixelsize': incaDictionary_ZS['Metadata']['DxInMetres'],
                     'ypixelsize': incaDictionary_ZS['Metadata']['DyInMetres'], 'x1': projection_bounds[0],
                     'y1': projection_bounds[1], 'x2': projection_bounds[2], 'y2': projection_bounds[3],
                     'yorigin': 'upper'}

Example 1: Output the precipitation type data from 2 members at all time steps

In [11]:
"""
Example 1
"""
# Choose your desired output. This choice will return a 3D array (time stamp, X-Coord, Y-Coord) for each member
desired_output = 'members'

# This array list will be a 4D array containing the precipitation type at each pixel for each time step for each member
array_list = postprocessor_diagnostics_prtype(filename=filename,
                                             startdate=startdate,
                                             snowLevelData=snowLevelData,
                                             temperatureData=temperatureData,
                                             groundTemperatureData=groundTemperatureData,
                                             modelMetadataDictionary=modelMetadataDictionary,
                                             topoFilename=topo,
                                             nwc_projectionString=nwc_projection_string,
                                             members=members,
                                             timeBase=60,
                                             timeStep=5,
                                             topo_interpolation=topo_interpolation,
                                             desired_output=desired_output)

Data load done
Topography load done
netCDF4 load done
Re-projection done
Interpolation done!
Calculate precipitation type per member over time...
Calculating precipitation types at:  2022-01-08 00:00:00
Calculating precipitation types at:  2022-01-08 00:05:00
Calculating precipitation types at:  2022-01-08 00:10:00
Calculating precipitation types at:  2022-01-08 00:15:00
Calculating precipitation types at:  2022-01-08 00:20:00
Calculating precipitation types at:  2022-01-08 00:25:00
Calculating precipitation types at:  2022-01-08 00:30:00
Calculating precipitation types at:  2022-01-08 00:35:00
Calculating precipitation types at:  2022-01-08 00:40:00
Calculating precipitation types at:  2022-01-08 00:45:00
Calculating precipitation types at:  2022-01-08 00:50:00
Calculating precipitation types at:  2022-01-08 00:55:00
Calculating precipitation types at:  2022-01-08 01:00:00
Calculating precipitation types at:  2022-01-08 01:05:00
Calculating precipitation types at:  2022-01-08 01:10:00

In [9]:
print(array_list.shape)

(2, 73, 591, 601)


Example 2: Output the mean precipitation type at every time step across all members

In [12]:
"""
Example 2
"""
# This time we will output the mean precipitation type across all members
desired_output = 'mean'

# This array list will be a 3D array containing the mean precipitation type at each pixel at each time step across all members
array_list_2 = postprocessor_diagnostics_prtype(filename=filename,
                                                 startdate=startdate,
                                                 snowLevelData=snowLevelData,
                                                 temperatureData=temperatureData,
                                                 groundTemperatureData=groundTemperatureData,
                                                 modelMetadataDictionary=modelMetadataDictionary,
                                                 topoFilename=topo,
                                                 nwc_projectionString=nwc_projection_string,
                                                 members=members,
                                                 timeBase=60,
                                                 timeStep=5,
                                                 topo_interpolation=topo_interpolation,
                                                 desired_output=desired_output)

Data load done
Topography load done
netCDF4 load done
Re-projection done
Interpolation done!
Calculate precipitation type per member over time...
Calculating precipitation types at:  2022-01-08 00:00:00
Calculating precipitation types at:  2022-01-08 00:05:00
Calculating precipitation types at:  2022-01-08 00:10:00
Calculating precipitation types at:  2022-01-08 00:15:00
Calculating precipitation types at:  2022-01-08 00:20:00
Calculating precipitation types at:  2022-01-08 00:25:00
Calculating precipitation types at:  2022-01-08 00:30:00
Calculating precipitation types at:  2022-01-08 00:35:00
Calculating precipitation types at:  2022-01-08 00:40:00
Calculating precipitation types at:  2022-01-08 00:45:00
Calculating precipitation types at:  2022-01-08 00:50:00
Calculating precipitation types at:  2022-01-08 00:55:00
Calculating precipitation types at:  2022-01-08 01:00:00
Calculating precipitation types at:  2022-01-08 01:05:00
Calculating precipitation types at:  2022-01-08 01:10:00

In [13]:
print(array_list_2)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 4. 4. 4.]
  [0. 0. 0. ... 4. 4. 4.]
  [0. 0. 0. ... 4. 4. 4.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 4. 4. 4.]
  [0. 0. 0. ... 4. 4. 4.]
  [0. 0. 0. ... 4. 4. 4.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 4. 4.]
  [0. 0. 0. ... 4. 4. 4.]]]


In [14]:
print(array_list_2.shape)

(73, 591, 601)


Example 3: Create plots and a gif of the precipitation type data from one of the ensemble members produced in Example 1

In [16]:
"""
Example 3
"""

import imageio
from datetime import timedelta
# In this example we will plot the precipitation type data from one of the ensemble members produced in Example 1.
# This example can be easily adjusted to plot the mean data instead.

# Specify the output directory
output_dir = "Add the path to your desired output directory"
# Choose the index of the member to plot.
member = 0
# Create an array of the time stamps
timestamp_idxs = []
time = startdate
for i in range(0, len(array_list[member])):
    timestamp_idxs.append(time)
    time += timedelta(minutes=timeStep)

# Plot members
filenames = []
for ts in range(len(array_list[member])):
    # Plot
    filenames.append(plot_ptype(array_list[member, ts, :, :], modelMetadataDictionary, ts, timestamp_idxs[ts], output_dir))

print("Plots created")
kargs = {'duration': 0.4}
with imageio.get_writer(
        os.path.join(output_dir, (
                str(member) + '_' + startdate.strftime('%Y%m%d%H%M') + '.gif')), mode='I',
        **kargs) as writer:
    for file in filenames:
        image = imageio.imread_v2(os.path.join(output_dir, file))
        writer.append_data(image)

# Close gif writer
writer.close()
print("Gif created")

Plots created
Gif created
